## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,78.80,89,75,4.61,moderate rain
1,1,Castro,BR,-24.7911,-50.0119,80.85,70,12,4.05,few clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,-21.50,68,73,2.80,broken clouds
3,3,Bonavista,CA,48.6499,-53.1147,37.00,100,100,4.61,light rain
4,4,Mataura,NZ,-46.1927,168.8643,51.78,67,54,5.26,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']>=min_temp) & (city_data_df['Max Temp']<=max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Bonavista,CA,48.6499,-53.1147,37.00,100,100,4.61,light rain
4,4,Mataura,NZ,-46.1927,168.8643,51.78,67,54,5.26,broken clouds
5,5,Ribeira Grande,PT,38.5167,-28.7000,53.60,66,40,26.46,scattered clouds
6,6,Kapaa,US,22.0752,-159.3190,68.00,88,75,8.05,broken clouds
8,8,Hermanus,ZA,-34.4187,19.2345,64.00,86,4,6.13,clear sky
9,9,Hobart,AU,-42.8794,147.3294,57.99,100,90,4.61,light intensity drizzle
12,12,Busselton,AU,-33.6500,115.3333,51.01,79,0,17.78,clear sky
16,16,Hofn,IS,64.2539,-15.2082,33.80,64,90,8.05,overcast clouds
17,17,Namibe,AO,-15.1961,12.1522,73.27,71,100,7.16,overcast clouds
18,18,Nangomba,TZ,-10.9000,38.5000,73.44,87,45,1.97,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bonavista,CA,37.00,light rain,48.6499,-53.1147,
4,Mataura,NZ,51.78,broken clouds,-46.1927,168.8643,
5,Ribeira Grande,PT,53.60,scattered clouds,38.5167,-28.7000,
6,Kapaa,US,68.00,broken clouds,22.0752,-159.3190,
8,Hermanus,ZA,64.00,clear sky,-34.4187,19.2345,
9,Hobart,AU,57.99,light intensity drizzle,-42.8794,147.3294,
12,Busselton,AU,51.01,clear sky,-33.6500,115.3333,
16,Hofn,IS,33.80,overcast clouds,64.2539,-15.2082,
17,Namibe,AO,73.27,overcast clouds,-15.1961,12.1522,
18,Nangomba,TZ,73.44,scattered clouds,-10.9000,38.5000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # combining coordinates under location for params dictionary
    params["location"] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bonavista,CA,37.00,light rain,48.6499,-53.1147,Abbott's Bed & Breakfast
4,Mataura,NZ,51.78,broken clouds,-46.1927,168.8643,Ellie's Villa
5,Ribeira Grande,PT,53.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
6,Kapaa,US,68.00,broken clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
8,Hermanus,ZA,64.00,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
...,...,...,...,...,...,...,...
682,Port Shepstone,ZA,74.93,overcast clouds,-30.7414,30.4550,My Den Beachfront B&B And Self-Catering
683,Rabo De Peixe,PT,53.60,few clouds,37.8000,-25.5833,Quinta das Giestas
685,Russell,US,55.40,clear sky,32.3502,-85.1999,Macs
687,Port Pirie,AU,69.80,light rain,-33.1833,138.0167,John Pirie Motor Inn


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description},{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))